In [199]:
import numpy as np
import pandas as pd
import os, sys
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')


%cd C:\emobility

C:\emobility


In [218]:
df_act = pd.read_csv(f'./dbs/agents/1_activity_plans.csv')


emission_factor = pd.read_csv(f'./dbs/emission/EmissionFactor.csv', delimiter=";")
PHEV = emission_factor[['TTW', 'WTW']][(emission_factor['Type']== 'Car_PHEV_bensin') | (emission_factor['Type']== 'Car_PHEV_diesel')].mean(axis=0)
emission_factor = emission_factor.append( PHEV, ignore_index=True ).drop([5, 6], axis=0)
emission_factor.iloc[6,0] = 'Car_PHEV'
emission_factor.replace(['Car_gasoline','Car_diesel','Car_ethanol','Car_gas','Car_el ','Car_PHEV'],[0,1,4,5,2,3], inplace=True)

car_distribution = pd.read_csv(f'./dbs/emission/CarDistribution.csv', delimiter=";")


car_distribution.loc[:,['Gasoline', 'Diesel', 'El', 'PHEV', 'Ethanol', 'Gas']] = car_distribution.loc[:,['Gasoline', 'Diesel', 'El', 'PHEV', 'Ethanol', 'Gas']].div(car_distribution['Total'], axis=0)

car_distribution = car_distribution.drop(['Total'], axis=1)



In [208]:
def carType_assignment(row, car_distribution):
    if row['deso'][:2] == '14':
        prob = car_distribution[['Gasoline', 'Diesel', 'El', 'PHEV', 'Ethanol', 'Gas']][car_distribution.Kod==int(row['deso'][:4])].values.tolist()
        pred = np.random.multinomial(1, prob[0], 1)
        carType = pred[0].tolist().index(1)
    else:
        # for those not in VG, use VG general distribution
        prob = [0.579957, 0.354470, 0.003693, 0.006844, 0.043670, 0.011365]
        pred = np.random.multinomial(1, prob, 1)
        carType = pred[0].tolist().index(1)

    return carType




In [209]:
df_act_car_segments = df_act.drop_duplicates(subset=['person'])

df_act_car_segments['carType'] = 100

df_act_car_segments['carType'] = df_act_car_segments.apply( lambda row: carType_assignment(row, car_distribution), axis=1)

In [99]:
#df_act = pd.merge(df_act, df_act_car_segments[["person", "carType"]], left_on= "person", right_on="person", how="left")

In [210]:
df_act_car_segments.carType.value_counts(normalize=True)

0    0.579758
1    0.354600
4    0.043687
5    0.011558
3    0.006821
2    0.003576
Name: carType, dtype: float64

In [211]:
import geopandas as gpd


road_network = gpd.read_file(f'./dbs/emission/road/5_road_network_with_slope.shp')

road_network = road_network[['link_id','geometry']]
road_network = road_network.reset_index()

DeSO = gpd.read_file(f'./dbs/emission/deso_statistik_shp/Fordon_region.shp')
DeSO = DeSO[['Deso','geometry']]
DeSO['deso_category'] = DeSO['Deso'].str[4]


DeSO = DeSO.to_crs(road_network.crs)

In [52]:
from spatial_operations import spatial_join


if __name__ == '__main__':
    road_network = spatial_join(road_network, DeSO)

In [53]:
#A DeSO in category A is mostly outside major population concentrations or urban areas. DeSO in category B is mostly located in a population concentration or agglomeration, but not in the municipality's central location. Category C includes DeSO, which is mostly located in the municipality's central location.

road_network_zonecross= road_network[road_network.duplicated(subset=['index'], keep=False)]


road_network_zonecross_mix = road_network_zonecross.groupby(['index']).agg({'deso_category': ' '.join}).reset_index()

road_network_zonecross_mix = road_network_zonecross_mix[(road_network_zonecross_mix.deso_category=="C A")|(road_network_zonecross_mix.deso_category=="A C")]



road_network = road_network.drop_duplicates(subset=['index'])

road_network['deso_category'][road_network['index'].isin(list(road_network_zonecross_mix['index']))]='M'


In [82]:
road_network_Noduplicate = road_network.copy()

road_network_Noduplicate['deso_category'] = road_network_Noduplicate['deso_category'].fillna('M')


In [83]:
road_network_Noduplicate = road_network_Noduplicate.groupby(['link_id']).agg({'deso_category': ' '.join}).reset_index()

In [84]:
def deso_categories(row):
    if len(row['deso_category'])==1:
        deso_category = row['deso_category']
    elif 'M' in row['deso_category']:
        deso_category = 'M'
    else:
        deso_category = "".join(set(row['deso_category']))
        if (len(deso_category)>1) & ("C" in deso_category) :
            deso_category = 'M'
        elif len(deso_category)>1  :
            deso_category = 'B'

    return deso_category


In [85]:
road_network_Noduplicate['deso_category'] = road_network_Noduplicate.apply( lambda row: deso_categories(row), axis=1)

In [86]:
#road_network_Noduplicate= pd.merge(road_network_Noduplicate, road_network[["link_id", "Deso"]], left_on= "link_id", right_on="link_id", how="left")
road_network_Noduplicate['deso_category'].value_counts(normalize=True)

C    0.453287
A    0.324249
B    0.199443
M    0.023021
Name: deso_category, dtype: float64

In [163]:
df_act_car_tra = pd.read_csv(f'./dbs/emission/paraset0.csv', delimiter=",")


In [164]:
df_act_car_tra.sort_values(["person", "seq"], inplace=True)

In [166]:
df_act_car_tra = pd.merge(df_act_car_tra[['person', 'seq', 'time', 'purpose', 'type', 'link', 'distance_driven']], road_network_Noduplicate[['link_id', 'deso_category']], left_on= "link", right_on="link_id", how="left")

In [192]:
aaa = df_act_car_tra.head(10000)


In [193]:
df_act_car_tra[df_act_car_tra.person==1971397]


,person,seq,time,purpose,type,link,distance_driven,link_id,deso_category,link_distance
0,1044313,0,30175.0,school,vehicle enters traffic,960412050001f,0.000000,960412050001f,C,0.000000
1,1044313,1,30176.0,school,left link,960412050001f,0.000000,960412050001f,C,0.000000
2,1044313,2,30191.0,school,left link,960412050001r,0.140481,960412050001r,C,0.140481
3,1044313,3,30202.0,school,left link,237476280006r,0.246943,237476280006r,C,0.106463
4,1044313,4,30220.0,school,left link,237476280003r,0.420595,237476280003r,C,0.173651
...,...,...,...,...,...,...,...,...,...,...
97,1044313,97,52570.0,home,left link,5833582590001f,16.745102,5833582590001f,C,0.304535
98,1044313,98,52577.0,home,left link,237476280000f,16.813195,237476280000f,C,0.068094
99,1044313,99,52595.0,home,left link,237476280003f,16.986846,237476280003f,C,0.173651
100,1044313,100,52606.0,home,left link,237476280006f,17.093309,237476280006f,C,0.106463


In [169]:
df_act_car_tra['distance_driven_shift'] = df_act_car_tra['distance_driven'].shift(-1)
df_act_car_tra['person_shift'] = df_act_car_tra['person'].shift(-1).fillna(0).astype('int')

df_act_car_tra['link_distance']=0

In [170]:
df_act_car_tra['link_distance'] = df_act_car_tra.apply( lambda row: row['distance_driven_shift']-row['distance_driven'] if row['person'] == row['person_shift'] else 0, axis=1)

In [171]:
# According to link distance (I have checked in Arcmap), shift rows.
df_act_car_tra['link_distance'] = df_act_car_tra['link_distance'].shift(1).fillna(0)

In [172]:
df_OTP = pd.read_csv(f'./dbs/agents/od_pairs_vg.csv')

df_OTP_last_activity = df_OTP.drop_duplicates(subset=['person'], keep='last')


In [173]:
df_OTP[df_OTP.person==10002928]


,person,act_id,act_purpose,lat_o,lng_o,lat_d,lng_d,origin,destination,Age,Gender,cycling_speed,part
809881,10002928,0,work,57.757010,12.053026,57.728697,11.996491,1480,1480,51,Male,19.655208,29
809882,10002928,1,other,57.728697,11.996491,57.719952,11.998955,1480,1480,51,Male,17.658943,29
809883,10002928,2,work,57.719952,11.998955,57.728697,11.996491,1480,1480,51,Male,19.655208,29
809884,10002928,3,home,57.728697,11.996491,57.757010,12.053026,1480,1480,51,Male,17.658943,29


In [174]:
df_act_last_activity = df_act_car_tra[df_act_car_tra.type == 'vehicle leaves traffic']

df_act_last_activity = df_act_last_activity.drop_duplicates(subset=['person','type'], keep='last')

In [175]:
df_act_last_activity = pd.merge(df_act_last_activity[['person','seq']], df_OTP_last_activity[["person", "act_purpose"]], left_on= "person", right_on="person", how="left")


In [176]:
df_act_car_tra = pd.merge(df_act_car_tra, df_act_last_activity, on= ["person",'seq'], how="left")

In [177]:
df_act_car_tra['purpose'] = np.where(pd.isna(df_act_car_tra.purpose), df_act_car_tra['act_purpose'], df_act_car_tra['purpose'])

In [178]:
df_act_car_tra.sort_values(["person", "seq"], inplace=True)

In [179]:
df_act_car_tra['purpose'].bfill(axis ='rows', inplace=True)


In [180]:
df_act_car_tra.drop(['person_shift', 'distance_driven_shift','act_purpose', 'link_id'], inplace=True, axis=1)

In [220]:
df_act_car_tra_bycategories = df_act_car_tra.groupby(['person','purpose','deso_category'])['link_distance'].sum().reset_index()


In [221]:
df_act_car_tra_bycategories = pd.merge(df_act_car_tra_bycategories, df_act_car_segments[["person", "carType"]], left_on= "person", right_on="person", how="left")

In [222]:
df_act_car_tra_bycategories = pd.merge(df_act_car_tra_bycategories, emission_factor[["Type", "TTW"]], left_on= "carType", right_on="Type", how="left")

In [223]:
print('a')

a
